In [1]:
# kiểm tra card GPU và CUDA (xem phiên bản CUDA)
!nvidia-smi
!nvcc --version || true

Fri Nov  7 03:04:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# # convert_questions.py
# import json
# import re
# from pathlib import Path
# from typing import List, Any, Optional

# INPUT_PATH = Path("/kaggle/input/kltn-1/kltn_dataset (3).json")   # <-- thay đường dẫn file nguồn nếu cần
# OUTPUT_PATH = Path("/kaggle/working/data_question.json")


# def clean_option_text(s: str) -> str:
#     """Loại bỏ tiền tố nhãn như (A)., A., A) , A  chỉ khi thực sự là nhãn.
#     Không xóa chữ đầu của câu nếu đó là phần nội dung (ví dụ 'Học')."""
#     if not isinstance(s, str):
#         return str(s)
#     # Chỉ xóa khi:
#     #  - "(A) " hoặc "(A)." dạng trong ngoặc
#     #  - "A." hoặc "A)" hoặc "A:" (chữ + dấu)
#     #  - "A " (chữ + khoảng trắng) — nhiều file dùng kiểu "A text" với khoảng trắng ngay sau chữ nhãn
#     pattern = r'^\s*(?:\([A-Za-z]\)\s*|[A-Za-z][\.\)\-:]\s*|[A-Za-z]\s+)'
#     cleaned = re.sub(pattern, '', s).strip()
#     return cleaned


# def is_true_from_raw(raw: Any) -> bool:
#     """Chuyển các dạng 'T','F','TRUE','FALSE','Đ','D','1','0','YES','NO' thành boolean."""
#     if isinstance(raw, bool):
#         return raw
#     s = str(raw).strip().upper()
#     if s in ("T", "TRUE", "Đ", "D", "1", "YES", "Y"):
#         return True
#     if s in ("F", "FALSE", "0", "N", "NO"):
#         return False
#     # fallback: nếu ký tự đầu là T -> True, nếu F -> False
#     if len(s) > 0:
#         if s[0] == "T":
#             return True
#         if s[0] == "F":
#             return False
#     return False


# def normalize_corrects(corrects: Any, n: int) -> List[Any]:
#     """
#     Trả về list length n của các giá trị raw của đáp án đúng/đúng-sai.
#     Hỗ trợ: list, chuỗi 'T,T,F,F', 'TTFF', 'T F F T', ...
#     """
#     if isinstance(corrects, list):
#         lst = corrects
#     elif isinstance(corrects, str):
#         s = corrects.strip()
#         if "," in s:
#             lst = [x.strip() for x in s.split(",") if x.strip() != ""]
#         else:
#             parts = s.split()
#             if len(parts) == n:
#                 lst = parts
#             elif len(s) == n:
#                 lst = list(s)
#             else:
#                 lst = list(s)
#     else:
#         lst = []

#     if len(lst) >= n:
#         return lst[:n]
#     else:
#         return lst + ["F"] * (n - len(lst))


# def parse_multiple_choice(item: dict) -> Optional[dict]:
#     """Parse một mục multiple_choice.

#     Nếu số option sau khi làm sạch < 3 thì trả về None (bỏ qua mục này) theo yêu cầu.
#     Nếu số option > 4 thì cũng bỏ qua (xóa câu hỏi) theo yêu cầu mới.
#     """
#     q_text = item.get("question", "").strip()
#     raw_options = item.get("options", [])
#     cleaned_options = []
#     for o in raw_options:
#         if isinstance(o, dict):
#             txt = o.get("text", "") or o.get("answer", "") or ""
#         else:
#             txt = o
#         cleaned = clean_option_text(txt)
#         # Bỏ các option rỗng sau khi clean
#         if isinstance(cleaned, str) and cleaned.strip() == "":
#             continue
#         cleaned_options.append(cleaned)

#     # Nếu số lựa chọn < 3 thì bỏ qua mục multiple choice
#     if len(cleaned_options) < 3:
#         print(f"Bỏ qua câu hỏi multiple_choice vì chỉ có {len(cleaned_options)} lựa chọn: '{q_text[:60]}'")
#         return None

#     # Nếu số lựa chọn > 4 thì bỏ qua mục multiple choice (yêu cầu bổ sung)
#     if len(cleaned_options) > 4:
#         print(f"Bỏ qua câu hỏi multiple_choice vì có {len(cleaned_options)} lựa chọn (>4): '{q_text[:60]}'")
#         return None

#     correct_letter = str(item.get("correct_answer", "")).strip().upper()
#     letter_to_index = {chr(ord('A') + i): i for i in range(26)}
#     correct_index = letter_to_index.get(correct_letter, None)
#     options_out = []
#     for i, opt in enumerate(cleaned_options):
#         is_correct = (correct_index is not None and i == correct_index)
#         options_out.append({"answer": opt, "isCorrect": bool(is_correct)})
#     return {"question": q_text, "options": options_out}


# def parse_true_false_block(block: dict):
#     """
#     Chuyển block main_question + sub_questions + correct_answers
#     -> danh sách các câu hỏi True/False riêng biệt.

#     Mỗi mục con (sub_question) sẽ trở thành 1 câu hỏi riêng:
#       - question: phần đề bài gốc (đã loại bỏ đoạn trong ngoặc dạng "(Sách giáo khoa...)") +
#         " Phát biểu sau: " + nội dung ý con (đặt trong ngoặc kép) + " là đúng hay sai" (nếu có ý con)
#       - options: hai lựa chọn 'Đúng' và 'Sai'
#       - isCorrect: dựa trên correct_answers tương ứng (True nếu đúng)

#     Trả về list các dict câu hỏi (có thể rỗng nếu không có sub_questions).
#     """
#     main_context = block.get("main_question", "").strip()
#     subs = block.get("sub_questions", [])
#     corrects_raw = block.get("correct_answers", [])

#     def remove_sach_parentheses(s: str) -> str:
#         """Loại bỏ các phần trong ngoặc có nội dung chứa "sách giáo khoa" (không phân biệt hoa thường)."""
#         if not s:
#             return s
#         out = ""
#         i = 0
#         L = len(s)
#         while i < L:
#             if s[i] == '(':
#                 j = s.find(')', i+1)
#                 if j == -1:
#                     out += s[i:]
#                     break
#                 inner = s[i+1:j]
#                 if 'sách giáo khoa' in inner.lower() or 'sach giao khoa' in inner.lower():
#                     i = j+1
#                     continue
#                 else:
#                     out += s[i:j+1]
#                     i = j+1
#             else:
#                 out += s[i]
#                 i += 1
#         return out.strip()

#     subs_list = []
#     if isinstance(subs, dict):
#         for k in sorted(subs.keys(), key=lambda x: str(x).upper()):
#             v = subs[k]
#             if isinstance(v, dict):
#                 txt = v.get("text", "") or v.get("answer", "") or ""
#             else:
#                 txt = v
#             subs_list.append(txt)
#     elif isinstance(subs, list):
#         for s in subs:
#             if isinstance(s, dict):
#                 txt = s.get("text", "") or s.get("answer", "") or ""
#             else:
#                 txt = str(s)
#             subs_list.append(txt)
#     else:
#         subs_list = []

#     n = len(subs_list)
#     if n == 0:
#         return []

#     corrects_norm = normalize_corrects(corrects_raw, n)

#     out_questions = []
#     main_clean = remove_sach_parentheses(main_context) if main_context else ""

#     for i, opt_text in enumerate(subs_list):
#         cleaned = clean_option_text(opt_text)
#         cleaned = remove_sach_parentheses(cleaned)
#         raw_cor = corrects_norm[i] if i < len(corrects_norm) else "F"
#         is_true = is_true_from_raw(raw_cor)

#         # Cấu trúc câu hỏi mới theo yêu cầu — đặt nội dung ý con trong ngoặc kép
#         if main_clean:
#             if cleaned:
#                 question_text = f"{main_clean} Phát biểu sau: \"{cleaned}\" là đúng hay sai"
#             else:
#                 question_text = f"{main_clean} là đúng hay sai"
#         else:
#             if cleaned:
#                 question_text = f"Phát biểu sau: \"{cleaned}\" là đúng hay sai"
#             else:
#                 question_text = "Phát biểu sau là đúng hay sai"

#         options_out = [
#             {"answer": "Đúng", "isCorrect": bool(is_true)},
#             {"answer": "Sai", "isCorrect": bool(not is_true)}
#         ]

#         out_questions.append({"question": question_text, "options": options_out})

#     return out_questions


# def convert(input_path: Path, output_path: Path):
#     with input_path.open("r", encoding="utf-8") as f:
#         data = json.load(f)

#     mc_converted = []
#     tf_converted = []

#     mc_list = data.get("multiple_choice", []) if isinstance(data, dict) else []
#     for item in mc_list:
#         try:
#             result = parse_multiple_choice(item)
#             if result is not None:
#                 mc_converted.append(result)
#             # nếu result is None thì đã in ra lý do ở trong parse_multiple_choice
#         except Exception as e:
#             print("Lỗi khi parse multiple_choice:", e)

#     tf_blocks = []
#     if isinstance(data, dict) and "true_false" in data and isinstance(data["true_false"], list):
#         tf_blocks.extend(data["true_false"])
#     else:
#         if isinstance(data, dict) and ("main_question" in data and "sub_questions" in data and "correct_answers" in data):
#             tf_blocks.append(data)

#     for blk in tf_blocks:
#         try:
#             result = parse_true_false_block(blk)
#             # parse_true_false_block trả về list các câu hỏi (hoặc rỗng)
#             if isinstance(result, list):
#                 tf_converted.extend(result)
#             elif result is not None:
#                 tf_converted.append(result)
#         except Exception as e:
#             print("Lỗi khi parse block true/false:", e)

#     output = {
#         "multiple_choice": mc_converted,
#         "true_false": tf_converted
#     }

#     with output_path.open("w", encoding="utf-8") as f:
#         json.dump(output, f, ensure_ascii=False, indent=2)

#     print(f"Đã chuyển xong: multiple_choice={len(mc_converted)}, true_false={len(tf_converted)}. Kết quả lưu tại: {output_path}\n")


# if __name__ == "__main__":
#     convert(INPUT_PATH, OUTPUT_PATH)


In [3]:
import os, sys, json, textwrap
try:
    import langchain
except Exception:
    # Một số package có thể đã có sẵn trên Kaggle; nếu không, cài
    !pip install -q langchain transformers sentence-transformers faiss-cpu accelerate bitsandbytes
    !pip install -q "huggingface_hub>=0.14.1"
    # vLLM optional (không bật tự động) - chỉ cài nếu bạn muốn thử vLLM
    !pip install -q vllm

In [4]:
!pip install faiss-cpu langchain-community "sentence-transformers" "faiss-cpu" transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.9/469.9 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56

In [5]:
pip install --upgrade transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 85.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.33.1
    Uninstalling huggingface-hub-0.33.1:
      Successfully uninstalled huggingface-hub-0.33.1
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled tr

In [6]:
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
import numpy as np
from typing import Dict, List, Any, Tuple
import time
from datetime import datetime
import re
from tqdm import tqdm
import faiss
from sklearn.preprocessing import normalize

class GraphRAGSystem:
    def __init__(self, kg_path: str, model_name: str = "Qwen/Qwen2.5-1.6B", 
                 embedding_model: str = "BAAI/bge-small-en-v1.5",
                 use_rerank: bool = False):  # Thêm option để tắt rerank nếu cần
        print("Initializing GraphRAG System...")
        
        # Load Knowledge Graph
        print("Loading knowledge graph...")
        self.kg = self.load_kg(kg_path)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.gpu_count = torch.cuda.device_count()
        self.use_rerank = use_rerank
        print(f"Using device: {self.device}")
        
        # Load models with progress tracking - use smaller models for speed
        print("Loading language model...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if not self.tokenizer.pad_token:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name, 
            torch_dtype=torch.float16,
            device_map="auto",
            trust_remote_code=True
        )
        
        print("Loading embedding model...")
        self.embedding_tokenizer = AutoTokenizer.from_pretrained(embedding_model, trust_remote_code=True)
        self.embedding_model = AutoModel.from_pretrained(
            embedding_model,
            torch_dtype=torch.float16,
            device_map="auto",
            trust_remote_code=True
        )
        
        # Only load rerank model if needed
        self.rerank_model = None
        self.rerank_tokenizer = None
        if use_rerank:
            print("Loading rerank model...")
            try:
                from transformers import AutoModelForSequenceClassification
                self.rerank_tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-reranker-small", trust_remote_code=True)
                if not self.rerank_tokenizer.pad_token:
                    self.rerank_tokenizer.pad_token = self.rerank_tokenizer.eos_token
                    
                self.rerank_model = AutoModelForSequenceClassification.from_pretrained(
                    "BAAI/bge-reranker-small",
                    torch_dtype=torch.float16,
                    device_map="auto",
                    trust_remote_code=True
                )
            except Exception as e:
                print(f"Failed to load rerank model: {e}. Continuing without reranking.")
                self.use_rerank = False
        
        # Build entity and text index with progress tracking
        print("Building entity index...")
        self.entity_index = self.build_entity_index()
        
        print("Preparing text chunks...")
        self.text_chunks = self.prepare_text_chunks()
        
        print("Computing chunk embeddings...")
        self.chunk_embeddings, self.faiss_index = self.compute_chunk_embeddings_faiss()
        
        print("GraphRAG System initialized successfully!")
    
    def load_kg(self, kg_path: str) -> Dict:
        with open(kg_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    
    def build_entity_index(self) -> Dict[str, Any]:
        entity_index = {}
        for entity in self.kg.get("entities", []):
            entity_id = entity["id"]
            entity_index[entity_id] = entity
            # Add labels to index
            for label in entity.get("label", []):
                entity_index[label.lower()] = entity
        return entity_index
    
    def prepare_text_chunks(self) -> List[Dict]:
        chunks = []
        chunk_id = 0
        
        # Create chunks from entities
        for entity in self.kg.get("entities", []):
            # Giới hạn độ dài text để tăng tốc embedding
            text = f"{entity.get('description', '')} {entity.get('original_text', '')}"
            if len(text) > 500:  # Giới hạn độ dài
                text = text[:500] + "..."
                
            chunk = {
                "id": chunk_id,
                "text": text,
                "entity_id": entity["id"],
                "type": "entity"
            }
            chunks.append(chunk)
            chunk_id += 1
        
        # Create chunks from triplets
        for triplet in self.kg.get("triplets", []):
            subject = self.entity_index.get(triplet["subject_id"], {})
            object_ent = self.entity_index.get(triplet["object_id"], {})
            
            chunk_text = f"{subject.get('label', [''])[0]} {triplet['predicate']} {object_ent.get('label', [''])[0]}"
            chunk = {
                "id": chunk_id,
                "text": chunk_text,
                "triplet": triplet,
                "type": "relation"
            }
            chunks.append(chunk)
            chunk_id += 1
            
        print(f"Created {len(chunks)} text chunks")
        return chunks
    
    def compute_chunk_embeddings_faiss(self) -> Tuple[torch.Tensor, faiss.Index]:
        """Compute embeddings using batch processing and create FAISS index for fast search"""
        embeddings = []
        batch_size = 64  # Tăng batch size để tăng tốc
        
        print(f"    Computing embeddings for {len(self.text_chunks)} chunks in batches of {batch_size}...")
        
        # Process in batches
        for i in tqdm(range(0, len(self.text_chunks), batch_size), desc="Embedding chunks"):
            batch_chunks = self.text_chunks[i:i+batch_size]
            batch_texts = [chunk["text"] for chunk in batch_chunks]
            
            inputs = self.embedding_tokenizer(
                batch_texts, 
                padding=True, 
                truncation=True, 
                max_length=256,  # Giảm max length để tăng tốc
                return_tensors="pt"
            ).to(self.device)
            
            with torch.no_grad():
                outputs = self.embedding_model(**inputs)
                # Use [CLS] token or mean pooling
                if hasattr(outputs, 'last_hidden_state'):
                    batch_embeddings = outputs.last_hidden_state[:, 0, :]  # Use [CLS] token
                else:
                    batch_embeddings = outputs.pooler_output
                embeddings.append(batch_embeddings.cpu())
                
        all_embeddings = torch.cat(embeddings, dim=0)
        
        # Create FAISS index for fast similarity search
        embeddings_np = all_embeddings.numpy().astype('float32')
        embeddings_np = normalize(embeddings_np, axis=1, norm='l2')  # Normalize for cosine similarity
        
        dimension = embeddings_np.shape[1]
        faiss_index = faiss.IndexFlatIP(dimension)  # Inner product for cosine similarity
        faiss_index.add(embeddings_np)
        
        return all_embeddings, faiss_index
    
    def embed_text(self, text: str) -> torch.Tensor:
        inputs = self.embedding_tokenizer(
            text, 
            padding=True, 
            truncation=True, 
            max_length=256,  # Giảm max length
            return_tensors="pt"
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.embedding_model(**inputs)
            if hasattr(outputs, 'last_hidden_state'):
                embedding = outputs.last_hidden_state[:, 0, :]  # Use [CLS] token
            else:
                embedding = outputs.pooler_output
            return embedding.cpu()
    
    def semantic_search(self, query: str, top_k: int = 5) -> List[Dict]:
        query_embedding = self.embed_text(query)
        query_embedding_np = query_embedding.numpy().astype('float32')
        query_embedding_np = normalize(query_embedding_np, axis=1, norm='l2')
        
        # FAISS search - much faster
        scores, indices = self.faiss_index.search(query_embedding_np, top_k * 2)  # Get more initially
        
        top_chunks = [self.text_chunks[i] for i in indices[0]]
        
        # Rerank only if enabled and model is available
        if self.use_rerank and self.rerank_model is not None:
            reranked_chunks = self.rerank_results_safe(query, top_chunks, top_k)
        else:
            # Just use FAISS scores
            scored_chunks = list(zip(top_chunks, scores[0]))
            scored_chunks.sort(key=lambda x: x[1], reverse=True)
            reranked_chunks = [chunk for chunk, score in scored_chunks[:top_k]]
        
        return reranked_chunks
    
    def rerank_results_safe(self, query: str, chunks: List[Dict], top_k: int) -> List[Dict]:
        """Safe reranking by processing one pair at a time"""
        if len(chunks) == 0:
            return []
            
        scores = []
        
        for chunk in tqdm(chunks, desc="Reranking", leave=False):
            try:
                # Process each query-chunk pair individually
                inputs = self.rerank_tokenizer(
                    query,
                    chunk["text"],
                    padding=True,
                    truncation=True,
                    max_length=256,
                    return_tensors="pt"
                ).to(self.device)
                
                with torch.no_grad():
                    outputs = self.rerank_model(**inputs)
                    score = torch.nn.functional.softmax(outputs.logits, dim=1)[0, 1].item()
                    scores.append(score)
            except Exception as e:
                print(f"Reranking error for chunk {chunk['id']}: {e}")
                scores.append(0.0)  # Default score if error
        
        # Sort by rerank scores
        scored_chunks = list(zip(chunks, scores))
        scored_chunks.sort(key=lambda x: x[1], reverse=True)
        
        return [chunk for chunk, score in scored_chunks[:top_k]]
    
    def analyze_question_fast(self, question: str) -> Dict:
        """Fast question analysis using simple rule-based approach instead of LLM"""
        # Simple rule-based analysis for speed
        question_lower = question.lower()
        
        # Extract keywords (simple approach)
        stop_words = {"là", "gì", "của", "trong", "vào", "từ", "đến", "với", "và", "hoặc", "các", "những"}
        words = [word for word in re.findall(r'\w+', question_lower) if word not in stop_words and len(word) > 1]
        keywords = " ".join(words[:5])  # Take top 5 keywords
        
        # Simple pattern matching for subject/action/object
        subject = ""
        action = ""
        obj = ""
        
        # Common patterns in Vietnamese questions
        patterns = {
            "ai": "person",
            "cái gì": "thing", 
            "ở đâu": "location",
            "khi nào": "time",
            "tại sao": "reason"
        }
        
        for pattern, category in patterns.items():
            if pattern in question_lower:
                action = category
                break
        
        return {
            "subject": subject,
            "action": action,
            "object": obj,
            "keywords": keywords,
            "full_analysis": f"Fast analysis - Keywords: {keywords}"
        }
    
    def retrieve_relevant_context(self, question: str, analysis: Dict) -> str:
        """Truy xuất ngữ cảnh liên quan từ KG"""
        # Search using full question
        chunks = self.semantic_search(question, top_k=3)
        
        # Also search using key components if available
        if analysis["keywords"] and analysis["keywords"].strip():
            keyword_chunks = self.semantic_search(analysis["keywords"], top_k=2)
            chunks.extend(keyword_chunks)
        
        # Remove duplicates
        seen_texts = set()
        unique_chunks = []
        for chunk in chunks:
            if chunk["text"] not in seen_texts:
                seen_texts.add(chunk["text"])
                unique_chunks.append(chunk)
        
        # Combine context - limit to top 3 for speed
        context_parts = []
        for chunk in unique_chunks[:3]:
            context_parts.append(chunk["text"])
            
        return "\n".join(context_parts) if context_parts else "Không tìm thấy thông tin liên quan."
    
    def answer_question(self, question: str, options: List[Dict], question_type: str) -> Dict:
        """Trả lời câu hỏi trắc nghiệm - optimized version"""
        start_time = time.time()
        
        # Fast question analysis
        analysis = self.analyze_question_fast(question)
        
        # Retrieve relevant context
        context = self.retrieve_relevant_context(question, analysis)
        
        # Prepare options text
        options_text = ""
        option_letters = ["A", "B", "C", "D"]
        for i, (option, letter) in enumerate(zip(options, option_letters)):
            options_text += f"{letter}. {option['answer']}\n"
        
        # Create optimized prompt
        if question_type == "multiple_choice":
            prompt = f"""Dựa trên thông tin sau, chọn đáp án đúng:

Thông tin: {context}

Câu hỏi: {question}

Lựa chọn:
{options_text}

Trả lời ngắn gọn: "Đáp án: [A/B/C/D]" """
        else:  # true_false
            prompt = f"""Xác định tính đúng/sai:

Thông tin: {context}

Câu hỏi: {question}

Lựa chọn:
{options_text}

Trả lời ngắn gọn: "Đáp án: [A/B]" """
        
        # Generate answer with shorter parameters
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=50,  # Rất ngắn
                do_sample=False,
                temperature=0.1,
                pad_token_id=self.tokenizer.eos_token_id
            )
        
        full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extract answer using regex
        answer_match = re.search(r"Đáp án\s*:\s*([ABCD])", full_response, re.IGNORECASE)
        predicted_answer = None
        confidence = 0.0
        
        if answer_match:
            predicted_letter = answer_match.group(1).upper()
            # Find the corresponding option
            for i, letter in enumerate(option_letters[:len(options)]):
                if letter == predicted_letter:
                    predicted_answer = options[i]['answer']
                    confidence = 0.8
                    break
        else:
            # Fallback: check if any option letter appears in response
            for i, letter in enumerate(option_letters[:len(options)]):
                if re.search(rf'\b{letter}\b', full_response, re.IGNORECASE):
                    predicted_answer = options[i]['answer']
                    confidence = 0.5
                    break
        
        processing_time = time.time() - start_time
        
        return {
            "predicted_answer": predicted_answer,
            "confidence": confidence,
            "full_response": full_response,
            "context": context,
            "analysis": analysis,
            "processing_time": processing_time
        }

class Evaluator:
    def __init__(self, rag_system: GraphRAGSystem):
        self.rag_system = rag_system
        self.results = {
            "multiple_choice": {"questions": [], "stats": {}},
            "true_false": {"questions": [], "stats": {}},
            "overall": {}
        }
    
    def evaluate_questions(self, questions_data: Dict, num_questions: int = None) -> Dict:
        print("Starting evaluation...")
        start_time = time.time()
        
        # Evaluate multiple choice questions
        mc_questions = questions_data.get("multiple_choice", [])
        if num_questions:
            mc_questions = mc_questions[:num_questions]
        
        print(f"Evaluating {len(mc_questions)} multiple choice questions...")
        self.evaluate_question_type(mc_questions, "multiple_choice")
        
        # Evaluate true/false questions
        tf_questions = questions_data.get("true_false", [])
        if num_questions:
            tf_questions = tf_questions[:num_questions]
        
        print(f"Evaluating {len(tf_questions)} true/false questions...")
        self.evaluate_question_type(tf_questions, "true_false")
        
        # Calculate overall statistics
        self.calculate_overall_stats()
        
        elapsed_time = time.time() - start_time
        total_questions = len(mc_questions) + len(tf_questions)
        self.results["overall"]["elapsed_time"] = f"{elapsed_time:.2f}s"
        self.results["overall"]["questions_per_minute"] = f"{(total_questions) / (elapsed_time / 60):.1f}"
        
        print("Evaluation completed!")
        return self.results
    
    def evaluate_question_type(self, questions: List[Dict], q_type: str):
        correct_count = 0
        wrong_count = 0
        no_prediction_count = 0
        total_processing_time = 0
        
        question_type_name = "Multiple Choice" if q_type == "multiple_choice" else "True/False"
        
        for i, q_data in enumerate(tqdm(questions, desc=f"{question_type_name}", unit="question")):
            question = q_data["question"]
            options = q_data["options"]
            
            # Get correct answer
            correct_answer = None
            for opt in options:
                if opt["isCorrect"]:
                    correct_answer = opt["answer"]
                    break
            
            # Get model prediction
            result = self.rag_system.answer_question(question, options, q_type)
            predicted_answer = result["predicted_answer"]
            total_processing_time += result.get("processing_time", 0)
            
            # Check if correct
            is_correct = predicted_answer == correct_answer if predicted_answer else False
            
            if predicted_answer is None:
                no_prediction_count += 1
            elif is_correct:
                correct_count += 1
            else:
                wrong_count += 1
            
            # Prepare options with markers (simplified for speed)
            option_letters = ["A", "B", "C", "D"]
            formatted_options = []
            for j, (opt, letter) in enumerate(zip(options, option_letters)):
                is_predicted = opt["answer"] == predicted_answer
                is_correct_opt = opt["isCorrect"]
                
                marker = ""
                if is_correct_opt and is_predicted:
                    marker = "(Correct + Predicted)"
                elif is_correct_opt:
                    marker = "(Correct)"
                elif is_predicted:
                    marker = "(Predicted)"
                
                formatted_options.append({
                    "letter": letter,
                    "text": opt["answer"],
                    "is_correct": is_correct_opt,
                    "is_predicted": is_predicted,
                    "marker": marker
                })
            
            # Store question result (minimal data)
            question_result = {
                "id": i + 1,
                "question": question,
                "options": formatted_options,
                "predicted_answer": predicted_answer or "[NO PREDICTION]",
                "correct_answer": correct_answer,
                "is_correct": is_correct,
                "confidence": result["confidence"],
                "processing_time": result.get("processing_time", 0)
            }
            
            self.results[q_type]["questions"].append(question_result)
            
            # Print progress every 5 questions
            if (i + 1) % 5 == 0:
                current_accuracy = (correct_count / (i + 1)) * 100
                avg_time = total_processing_time / (i + 1)
                tqdm.write(f"   Completed {i+1}/{len(questions)} - Accuracy: {current_accuracy:.1f}% - Avg time: {avg_time:.1f}s")
        
        # Calculate statistics
        total = len(questions)
        accuracy = (correct_count / total) * 100 if total > 0 else 0
        avg_processing_time = total_processing_time / total if total > 0 else 0
        
        self.results[q_type]["stats"] = {
            "total": total,
            "correct": correct_count,
            "wrong": wrong_count,
            "no_prediction": no_prediction_count,
            "accuracy": accuracy,
            "avg_processing_time": avg_processing_time
        }
        
        print(f"   {question_type_name} Results: {correct_count}/{total} correct ({accuracy:.1f}%) - Avg time: {avg_processing_time:.1f}s")
    
    def calculate_overall_stats(self):
        mc_stats = self.results["multiple_choice"]["stats"]
        tf_stats = self.results["true_false"]["stats"]
        
        total_questions = mc_stats["total"] + tf_stats["total"]
        total_correct = mc_stats["correct"] + tf_stats["correct"]
        total_no_prediction = mc_stats["no_prediction"] + tf_stats["no_prediction"]
        
        overall_accuracy = (total_correct / total_questions) * 100 if total_questions > 0 else 0
        prediction_rate = ((total_questions - total_no_prediction) / total_questions) * 100 if total_questions > 0 else 0
        
        self.results["overall"].update({
            "total_questions": total_questions,
            "total_correct": total_correct,
            "total_no_prediction": total_no_prediction,
            "overall_accuracy": overall_accuracy,
            "prediction_rate": prediction_rate
        })

    def print_summary(self, results: Dict):
        print(f"\n{'='*60}")
        print(f"{'FINAL RESULTS':^60}")
        print(f"{'='*60}")
        
        mc = results['multiple_choice']['stats']
        tf = results['true_false']['stats']
        ov = results['overall']
        
        print(f"\nMULTIPLE CHOICE: {mc['correct']}/{mc['total']} = {mc['accuracy']:.1f}%")
        print(f"  Avg time: {mc.get('avg_processing_time', 0):.1f}s per question")
        print(f"TRUE/FALSE:      {tf['correct']}/{tf['total']} = {tf['accuracy']:.1f}%")
        print(f"  Avg time: {tf.get('avg_processing_time', 0):.1f}s per question")
        print(f"\nOVERALL:         {ov['total_correct']}/{ov['total_questions']} = {ov['overall_accuracy']:.1f}%")
        print(f"TIME:            {ov['elapsed_time']}")
        print(f"SPEED:           {ov.get('questions_per_minute', 'N/A')} questions/minute")
        print(f"{'='*60}\n")

def main():
    # Initialize the RAG system
    print("Starting Optimized GraphRAG Evaluation System...")
    
    # Sử dụng model nhỏ và tắt rerank để tăng tốc
    rag_system = GraphRAGSystem(
        "/kaggle/input/kltn-1/graph_documents_v3.json",
        model_name="Qwen/Qwen3-4B",  # Smaller model
        embedding_model="Qwen/Qwen3-Embedding-0.6B",  # Faster embedding model
        use_rerank=False  # Tắt rerank để tránh lỗi
    )
    
    # Load questions
    print("Loading questions...")
    with open("/kaggle/input/kltn-1/question_1000.json", "r", encoding="utf-8") as f:
        questions_data = json.load(f)
    
    # Initialize evaluator
    evaluator = Evaluator(rag_system)
    
    # Evaluate questions
    print(f"Total questions loaded:")
    print(f"   - Multiple Choice: {len(questions_data.get('multiple_choice', []))}")
    print(f"   - True/False: {len(questions_data.get('true_false', []))}")
    
    # Bắt đầu với ít câu hỏi để test
    test_questions = 10  # Giảm xuống còn 3 để test nhanh
    print(f"\nEvaluating first {test_questions} questions of each type...")
    
    results = evaluator.evaluate_questions(questions_data)
    
    # Print summary
    evaluator.print_summary(results)
    
    print("Evaluation process completed!")

if __name__ == "__main__":
    main()

Starting Optimized GraphRAG Evaluation System...
Initializing GraphRAG System...
Loading knowledge graph...
Using device: cuda
Loading language model...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

   Completed 715/724 - Accuracy: 66.0% - Avg time: 4.1s


True/False:  99%|█████████▉| 720/724 [49:20<00:16,  4.05s/question]

   Completed 720/724 - Accuracy: 65.8% - Avg time: 4.1s


True/False: 100%|██████████| 724/724 [49:36<00:00,  4.11s/question]

   True/False Results: 477/724 correct (65.9%) - Avg time: 4.1s
Evaluation completed!

                       FINAL RESULTS                        

MULTIPLE CHOICE: 185/356 = 52.0%
  Avg time: 4.1s per question
TRUE/FALSE:      477/724 = 65.9%
  Avg time: 4.1s per question

OVERALL:         662/1080 = 61.3%
TIME:            4424.14s
SPEED:           14.6 questions/minute

Evaluation process completed!
